This notebook's model was inspired by the 1dcnn in https://github.com/baosenguo/Kaggle-MoA-2nd-Place-Solution from the excellent competitor https://www.kaggle.com/baosenguo Check out his page and work, he's very helpful


Also many thanks to Dr. Ahmed Sami for sharing his model and training flow in https://www.kaggle.com/ahmedelhaddad/fastai-submission-custom-ds! The only thing I changed was the output to a tensor of 5 items, to do BCEWithLogitsLoss https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html

This helped improve the model in all scenarios, and I will explain my reasoning for this and trials in the discussion forum soon!

Here are different models I've tried(here are the links and scores)

https://www.kaggle.com/pyoungkangkim/mlp-batchnorm-dropout-linear-pytorch-no-pre-bnorm

https://www.kaggle.com/pyoungkangkim/mlp-batchnorm-dropout-linear-pytorch

https://www.kaggle.com/pyoungkangkim/resnet-batchnorm-dropout-linear-pytorch-jstreet

https://www.kaggle.com/pyoungkangkim/densenet-batchnorm-dropout-linear-pytorch-jstreet

More to come below, I'll eventually release them(soon) once I submit them and get the scores.

https://www.kaggle.com/pyoungkangkim/resnet-deep-bnorm-dropout-linear-pytorch-jstreet

The particulary interesting ones to come(I speculate sometime mid Feburary) will be ensemble of these models and xgboost, check later on my profile if you're interested for this and trials/findings!

In [ ]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import datatable as dtable
import fastai

from fastai.tabular.all import *
from fastai.data.core import DataLoader
from fastai.data.core import DataLoaders
from numba import njit
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from time import time

df = dtable.fread('/kaggle/input/jane-street-market-prediction/train.csv').to_pandas()
df = df.query('date > 85').reset_index(drop = True) 
df = df[df.weight > 0]
df.reset_index(inplace=True, drop=True)
df = df.astype({c: np.float32 for c in df.select_dtypes(include='float64').columns})
df_labels = df[['date', 'weight', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']]
features = [c for c in df.columns if 'feature' in c]
df = df.drop(df_labels.columns, axis=1)
df.fillna(df.mean(), inplace=True)
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']
X = df[features]
y = np.stack([(df_labels[c] > 0).astype('int') for c in resp_cols]).T
f_mean = np.mean(df[features[1:]].values, axis=0)
device = 'cuda' if t.cuda.is_available() else 'cpu'

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        cha_1 = 64
        cha_2 = 128
        cha_3 = 128

        cha_1_reshape = int(hidden_size / cha_1)
        cha_po_1 = int(hidden_size / cha_1 / 2)
        cha_po_2 = int(hidden_size / cha_1 / 2 / 2) * cha_3

        self.cha_1_reshape = cha_1_reshape
        self.cha_1 = cha_1
        self.cha_2 = cha_2
        self.cha_3 = cha_3

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

        self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
        self.dropout_c1 = nn.Dropout(0.1)
        self.conv1 = nn.utils.weight_norm(nn.Conv1d(cha_1, cha_2, kernel_size=5, stride=1, padding=2, bias=False), dim=None)
        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size=cha_po_1)

        self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2 = nn.Dropout(0.1)
        self.conv2 = nn.utils.weight_norm(nn.Conv1d(cha_2, cha_2, kernel_size=3, stride=1, padding=1, bias=True), dim=None)

        self.batch_norm_c2_1 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_1 = nn.Dropout(0.3)
        self.conv2_1 = nn.utils.weight_norm(nn.Conv1d(cha_2, cha_2, kernel_size=3, stride=1, padding=1, bias=True), dim=None)

        self.batch_norm_c2_2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_2 = nn.Dropout(0.2)
        self.conv2_2 = nn.utils.weight_norm(nn.Conv1d(cha_2, cha_3, kernel_size=5, stride=1, padding=2, bias=True), dim=None)

        self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        self.batch_norm3 = nn.BatchNorm1d(cha_po_2)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))


    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.celu(self.dense1(x), alpha=0.06)

        x = x.reshape(x.shape[0], self.cha_1, self.cha_1_reshape)

        x = self.batch_norm_c1(x)
        x = self.dropout_c1(x)
        x = F.relu(self.conv1(x))

        x = self.ave_po_c1(x)

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = F.relu(self.conv2(x))
        x_s = x

        x = self.batch_norm_c2_1(x)
        x = self.dropout_c2_1(x)
        x = F.relu(self.conv2_1(x))

        x = self.batch_norm_c2_2(x)
        x = self.dropout_c2_2(x)
        x = F.relu(self.conv2_2(x))
        x = x * x_s

        x = self.max_po_c2(x)

        x = self.flt(x)

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)

        return x

In [ ]:
model_nn = Model(num_features=130, num_targets=5, hidden_size=512)
model_nn = model_nn.to(device)

In [ ]:
learn = Learner(None, model_nn, loss_func=1)
learn.load('/kaggle/input/jstreet1dcnn/1dcnn')

In [ ]:
@njit
def fillna_npwhere_njit(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

def for_loop(method, matrix, values):
    for i in range(matrix.shape[0]):
        matrix[i] = method(matrix[i], values)
    return matrix

In [ ]:
%%time
%%capture
import janestreet
env = janestreet.make_env()

learn.model.eval()
preds = []
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:   
        x_tt = test_df.loc[:, features].values
        x_tt[:, 1:] = for_loop(fillna_npwhere_njit, x_tt[:, 1:], f_mean)
        pred = 0.
        pred = learn.model(t.from_numpy(x_tt).to(device, t.float)).sigmoid()[0][-1].item()
        preds.append(pred)
        action = 1 if pred >= .5 else 0
        pred_df.action = action
    else:
        pred_df.action = 0
    env.predict(pred_df)

In [ ]:
preds = np.array(preds)
preds.mean(), preds.std(), sum(preds >= .5), sum(preds < 5)
